In [ ]:
!pip install geopandas
!pip install h3
!pip install shapely
!pip install folium
!pip install geopy
!pip install mapclassify

In [2]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

# tratando censo 2010

In [4]:
UF = ['AM', 'BA', 'MG', 'MS', 'MT', 'PA']

In [5]:
# Domicilio01
f = 'Domicilio01'

list_domicilio01 = [pd.read_csv(f'/content/drive/MyDrive/07. Regiao/{u}/CSV/{f}_{u}.csv', sep=';') for u in UF]

domicilio01 = pd.concat(list_domicilio01).reset_index(drop=True)
domicilio01 = domicilio01[['Cod_setor', 'V001']].copy()

domicilio01['Cod_setor'] = domicilio01['Cod_setor'].astype('str')
domicilio01['V001'] = domicilio01['V001'].astype('float')

domicilio01.columns = ['cod_setor', 'dom_total']

domicilio01['cod_uf'] = domicilio01['cod_setor'].str[:2]

domicilio01 = domicilio01[['cod_setor', 'cod_uf', 'dom_total']].copy()

print(domicilio01.shape)
print(domicilio01.isna().sum())
domicilio01.head()


(80892, 3)
cod_setor    0
cod_uf       0
dom_total    0
dtype: int64


,cod_setor,cod_uf,dom_total
0,130002905000001,13,149.0
1,130002905000002,13,199.0
2,130002905000003,13,147.0
3,130002905000004,13,23.0
4,130002905000006,13,103.0


In [6]:
# Pessoa01
f = 'Pessoa01'

list_pessoa01 = [pd.read_csv(f'/content/drive/MyDrive/07. Regiao/{u}/CSV/{f}_{u}.csv', sep=';') for u in UF]

pessoa01 = pd.concat(list_pessoa01).reset_index(drop=True)
pessoa01 = pessoa01[['Cod_setor', 'V001']].copy()

pessoa01['Cod_setor'] = pessoa01['Cod_setor'].astype('str')
pessoa01['V001'] = pessoa01['V001'].astype('float')

pessoa01.columns = ['cod_setor', 'pop_alfabetizada']

print(pessoa01.shape)
print(pessoa01.isna().sum())
pessoa01.head()


(80891, 2)
cod_setor           0
pop_alfabetizada    0
dtype: int64


,cod_setor,pop_alfabetizada
0,130002905000001,610.0
1,130002905000002,807.0
2,130002905000003,648.0
3,130002905000004,53.0
4,130002905000006,291.0


In [7]:
# Pessoa13
f = 'Pessoa13'

list_pessoa13 = [pd.read_csv(f'/content/drive/MyDrive/07. Regiao/{u}/CSV/{f}_{u}.csv', sep=';') for u in UF]

pessoa13_temp = pd.concat(list_pessoa13).reset_index(drop=True)

pessoa13_temp['V001'] = pd.to_numeric(pessoa13_temp['V001'], errors='coerce')
pessoa13_temp['V022'] = pd.to_numeric(pessoa13_temp['V022'], errors='coerce')

pessoa13_temp.iloc[:, 36:136] = pessoa13_temp.iloc[:, 36:136].apply(pd.to_numeric, errors='coerce')

pessoa13 = pd.DataFrame()
pessoa13['cod_setor'] = pessoa13_temp['Cod_setor'].astype('str')
pessoa13['pop_total'] = pessoa13_temp['V001'].astype('float')

pessoa13['pop_0_17_anos'] = (np.sum(pessoa13_temp.iloc[:,36:53], axis=1) + pessoa13_temp['V022']).astype('float')
pessoa13['pop_18_24_anos'] = np.sum(pessoa13_temp.iloc[:,53:60], axis=1).astype('float')
pessoa13['pop_25_34_anos'] = np.sum(pessoa13_temp.iloc[:,60:70], axis=1).astype('float')
pessoa13['pop_35_44_anos'] = np.sum(pessoa13_temp.iloc[:,70:80], axis=1).astype('float')
pessoa13['pop_45_59_anos'] = np.sum(pessoa13_temp.iloc[:,80:95], axis=1).astype('float')
pessoa13['pop_60_79_anos'] = np.sum(pessoa13_temp.iloc[:,95:115], axis=1).astype('float')
pessoa13['pop_80_99_anos'] = np.sum(pessoa13_temp.iloc[:,115:136], axis=1).astype('float')


print(pessoa13.shape)
print(pessoa13.isna().sum())
pessoa13.head()

(80891, 9)
cod_setor            0
pop_total            0
pop_0_17_anos     1513
pop_18_24_anos       0
pop_25_34_anos       0
pop_35_44_anos       0
pop_45_59_anos       0
pop_60_79_anos       0
pop_80_99_anos       0
dtype: int64


,cod_setor,pop_total,pop_0_17_anos,pop_18_24_anos,pop_25_34_anos,pop_35_44_anos,pop_45_59_anos,pop_60_79_anos,pop_80_99_anos
0,130002905000001,806.0,387.0,122.0,118.0,74.0,70.0,30.0,5.0
1,130002905000002,1098.0,565.0,154.0,185.0,94.0,76.0,23.0,1.0
2,130002905000003,807.0,362.0,111.0,98.0,80.0,91.0,56.0,9.0
3,130002905000004,118.0,62.0,13.0,18.0,10.0,5.0,10.0,0.0
4,130002905000006,564.0,302.0,58.0,81.0,48.0,48.0,22.0,5.0


In [8]:
# PessoaRenda
f = 'PessoaRenda'

list_pessoarenda = [pd.read_csv(f'/content/drive/MyDrive/07. Regiao/{u}/CSV/{f}_{u}.csv', sep=';') for u in UF]

pessoarenda = pd.concat(list_pessoarenda).reset_index(drop=True)

pessoarenda = pessoarenda.iloc[:,:24].copy().drop(columns = ['V020', 'V021', 'Situacao_setor', 'V010'])

pessoarenda['Cod_setor'] = pessoarenda['Cod_setor'].astype('str')
pessoarenda.iloc[:, 1:20] = pessoarenda.iloc[:, 1:20].apply(pd.to_numeric, errors='coerce')

for c in pessoarenda.columns:
  if c != 'Cod_setor':
    pessoarenda[c] = pessoarenda[c].astype('float')

pessoarenda.columns = ['cod_setor',
                       'pop_renda_0_1s2', 'pop_renda_1s2_1', 'pop_renda_1_2', 'pop_renda_2_3', 'pop_renda_3_5', 'pop_renda_5_10',
                       'pop_renda_10_15', 'pop_renda_15_20', 'pop_renda_20_99',
                       'renda_nom_0_1s2', 'renda_nom_1s2_1', 'renda_nom_1_2', 'renda_nom_2_3', 'renda_nom_3_5', 'renda_nom_5_10',
                       'renda_nom_10_15', 'renda_nom_15_20', 'renda_nom_20_99',
                       'renda_nom_total']

# pessoarenda['pop_total'] = pessoarenda['V001'].astype('float')

# pessoarenda['pop_0_17_anos'] = (np.sum(pessoarenda.iloc[:,36:53], axis=1) + pessoarenda['V022']).astype('float')
# pessoarenda['pop_18_24_anos'] = np.sum(pessoarenda.iloc[:,53:60], axis=1).astype('float')
# pessoarenda['pop_25_34_anos'] = np.sum(pessoarenda.iloc[:,60:70], axis=1).astype('float')
# pessoarenda['pop_35_44_anos'] = np.sum(pessoarenda.iloc[:,70:80], axis=1).astype('float')
# pessoarenda['pop_45_59_anos'] = np.sum(pessoarenda.iloc[:,80:95], axis=1).astype('float')
# pessoarenda['pop_60_79_anos'] = np.sum(pessoarenda.iloc[:,95:115], axis=1).astype('float')
# pessoarenda['pop_80_99_anos'] = np.sum(pessoarenda.iloc[:,115:136], axis=1).astype('float')


print(pessoarenda.shape)
print(pessoarenda.isna().sum())
pessoarenda.head()

(80892, 20)
cod_setor             0
pop_renda_0_1s2    1514
pop_renda_1s2_1    1514
pop_renda_1_2      1514
pop_renda_2_3      1514
pop_renda_3_5      1514
pop_renda_5_10     1514
pop_renda_10_15    1514
pop_renda_15_20    1514
pop_renda_20_99    1514
renda_nom_0_1s2    1514
renda_nom_1s2_1    1514
renda_nom_1_2      1514
renda_nom_2_3      1514
renda_nom_3_5      1514
renda_nom_5_10     1514
renda_nom_10_15    1514
renda_nom_15_20    1514
renda_nom_20_99    1514
renda_nom_total    1514
dtype: int64


,cod_setor,pop_renda_0_1s2,pop_renda_1s2_1,pop_renda_1_2,pop_renda_2_3,pop_renda_3_5,pop_renda_5_10,pop_renda_10_15,pop_renda_15_20,pop_renda_20_99,renda_nom_0_1s2,renda_nom_1s2_1,renda_nom_1_2,renda_nom_2_3,renda_nom_3_5,renda_nom_5_10,renda_nom_10_15,renda_nom_15_20,renda_nom_20_99,renda_nom_total
0,130002905000001,15.0,60.0,50.0,13.0,7.0,5.0,2.0,0.0,1.0,2123.0,27862.0,36303.0,17610.0,14200.0,17800.0,13730.0,0.0,13000.0,142628.0
1,130002905000002,74.0,145.0,62.0,15.0,15.0,1.0,1.0,1.0,0.0,11094.0,62453.0,45206.0,18784.0,29260.0,2800.0,6000.0,10000.0,0.0,185597.0
2,130002905000003,3.0,124.0,56.0,26.0,11.0,4.0,0.0,0.0,0.0,380.0,63000.0,49109.0,32780.0,21650.0,12000.0,0.0,0.0,0.0,178919.0
3,130002905000004,0.0,10.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4470.0,7944.0,0.0,0.0,0.0,0.0,0.0,0.0,12414.0
4,130002905000006,17.0,70.0,44.0,7.0,3.0,0.0,0.0,0.0,0.0,2264.0,32564.0,31958.0,9330.0,5150.0,0.0,0.0,0.0,0.0,81266.0


In [9]:
censo2010 = domicilio01.merge(
    pessoa01, on='cod_setor', how='left'
).merge(
    pessoa13, on='cod_setor', how='left'
).merge(
    pessoarenda, on='cod_setor', how='left'
)

censo2010.reset_index(drop=True, inplace=True)

print(censo2010.shape)
censo2010.head()

(80892, 31)


,cod_setor,cod_uf,dom_total,pop_alfabetizada,pop_total,pop_0_17_anos,pop_18_24_anos,pop_25_34_anos,pop_35_44_anos,pop_45_59_anos,...,renda_nom_0_1s2,renda_nom_1s2_1,renda_nom_1_2,renda_nom_2_3,renda_nom_3_5,renda_nom_5_10,renda_nom_10_15,renda_nom_15_20,renda_nom_20_99,renda_nom_total
0,130002905000001,13,149.0,610.0,806.0,387.0,122.0,118.0,74.0,70.0,...,2123.0,27862.0,36303.0,17610.0,14200.0,17800.0,13730.0,0.0,13000.0,142628.0
1,130002905000002,13,199.0,807.0,1098.0,565.0,154.0,185.0,94.0,76.0,...,11094.0,62453.0,45206.0,18784.0,29260.0,2800.0,6000.0,10000.0,0.0,185597.0
2,130002905000003,13,147.0,648.0,807.0,362.0,111.0,98.0,80.0,91.0,...,380.0,63000.0,49109.0,32780.0,21650.0,12000.0,0.0,0.0,0.0,178919.0
3,130002905000004,13,23.0,53.0,118.0,62.0,13.0,18.0,10.0,5.0,...,0.0,4470.0,7944.0,0.0,0.0,0.0,0.0,0.0,0.0,12414.0
4,130002905000006,13,103.0,291.0,564.0,302.0,58.0,81.0,48.0,48.0,...,2264.0,32564.0,31958.0,9330.0,5150.0,0.0,0.0,0.0,0.0,81266.0


In [10]:
censo2010.to_parquet('/content/drive/MyDrive/07. Regiao/censo2010.parquet')

# criar hex_grid_UF

In [11]:
import pandas as pd
import numpy as np

import geopandas as gpd
from shapely.geometry import Polygon, mapping
import h3
import folium

In [12]:
df_mun = gpd.read_file('/content/drive/MyDrive/07. Regiao/BR_Municipios_2022/BR_Municipios_2022.shp')
df_mun.shape

(5572, 5)

In [13]:
def list_h3_from_geometry(g, res):
  hex_list = []

  try:
    temp = mapping(g)

    # gets coordinates of geometry from the geojson
    temp['coordinates'] = [[[j[1], j[0]] for j in i] for i in temp['coordinates']]
    # temp['coordinates'] = [[j[1], j[0]] for j in temp['coordinates']]

    # fills polygon with hexagons that are contained by the geojson-like structure
    hex_list.extend(h3.polyfill(geojson = temp, res = res))

  except:

    for g in g.geoms:
      temp = mapping(g)

      # gets coordinates of geometry from the geojson
      temp['coordinates'] = [[j[1], j[0]] for j in temp['coordinates']]

      # fills polygon with hexagons that are contained by the geojson-like structure
      hex_list.extend(h3.polyfill(geojson = temp, res = res))


  return set(hex_list)



def criar_df_hex(df_mun):
  df = df_mun.copy()

  df['list_h3_10'] = df['geometry'].apply(lambda x: list_h3_from_geometry(x, 10))

  n = 0
  for i in df['list_h3_10'].values:
    n = n + len(i)

  assert len(set.union(*df['list_h3_10'].values)) == n

  df = df.drop(columns = ['geometry', 'AREA_KM2'])\
        .explode('list_h3_10')\
        .rename(columns = {'list_h3_10': 'h3_10_id'})\
        .reset_index(drop=True)

  return df


In [14]:
UF = ['MS', 'BA', 'MG', 'MT', 'PA', 'AM']


In [ ]:
for u in UF:
  print(f'{u} iniciando', 60*'-')
  df_hex = criar_df_hex(df_mun[df_mun['SIGLA_UF'] == u])

  print('df_hex criado com sucesso')

  df_hex.to_parquet(f'/content/drive/MyDrive/07. Regiao/hex_municipal/df_hex_{u}.parquet')
  print(f'{u} salvo com {df_hex.shape[0]} poligonos', 50*'-')


MS iniciando ------------------------------------------------------------
df_hex criado com sucesso
MS salvo com 23897642 poligonos --------------------------------------------------
BA iniciando ------------------------------------------------------------
